# Testing the order of points after tetgen.Tetrahedralize

The purpose of this file is to see if the order of points is perserved after tetgen.Tetrahedralize. This is important for computing the cat cells of different objects.


In [1]:
import numpy as np
import pyvista as pv
import tetgen

# Create a simple mesh
mesh1 = pv.Cube().clean().triangulate()
mesh2 = pv.Cube(center=(3, 0, 0)).clean().triangulate()
mesh3 = pv.Cube(center=(0, 3, 0)).clean().triangulate()
mesh4 = pv.Cone(center=(0, 0, 3)).clean().triangulate()
mesh5 = pv.Icosahedron(radius=9).clean().triangulate()

meshes = [mesh1, mesh2, mesh3, mesh4, mesh5]

# Merge the meshes
merged = pv.PolyData()
for mesh in meshes:
    merged.merge(mesh, inplace=True, merge_points=False)

tetmesh = tetgen.TetGen(merged)
tetmesh.tetrahedralize(order=1, mindihedral=0, minratio=0, quality=False, cdt=True)

(tetmesh.grid.celltypes == np.full(tetmesh.grid.n_cells, 10, dtype=np.uint8)).all()
id0 = 0
for i, mesh in enumerate(meshes):
    tet_points = tetmesh.grid.points[id0:id0 + mesh.n_points]

    mesh_points = mesh.points[0:mesh.n_points]
    # print(f"points tet and mesh: \n{tet_points}\n{mesh_points}\n")
    print(f"for mesh {i} ({mesh.n_points} points):")
    print((tet_points == mesh_points).all())

    id0 += mesh.n_points

for mesh 0 (8 points):
True
for mesh 1 (8 points):
True
for mesh 2 (8 points):
True
for mesh 3 (7 points):
True
for mesh 4 (12 points):
True


In [7]:
# if i have a list of meshes, i want to be able to check the ids of
# the points in a tetrahedron(cell of the tetmesh)
# and check if they are the same as the ids of the points in the mesh
cells = tetmesh.grid.cells
print(f"cell shape: {cells.shape}")
for cell in np.hsplit(cells, cells.size / (4 + 1)):
    assert cell[0] == 4
    print(cell)

cell shape: (875,)
[ 4 22 16 21 20]
[ 4  8 15 12 14]
[ 4 33 18 17 16]
[ 4 19 16 21 18]
[ 4 24 25 30 26]
[ 4  5 20 11 31]
[ 4 22 18 21 16]
[ 4 35 24 30 29]
[ 4 26 29 30 24]
[ 4 16  2 23 17]
[ 4 36  2  1  0]
[ 4 31 19 32 33]
[ 4 39 31  8 11]
[ 4 16 19 33 18]
[ 4 40 19 33 16]
[ 4  3 40  2 16]
[ 4  3  0  2 40]
[ 4 26 29 24 36]
[ 4 42  0 38 40]
[ 4 29 28  1 36]
[ 4 37 32 39 13]
[ 4  7  9  6 30]
[ 4 36 25 24 26]
[ 4 20  5 11 10]
[ 4 11 13 39 31]
[ 4 36  0  1 42]
[ 4 11 10  8 13]
[ 4 36 26 34 25]
[ 4 23  6  5 10]
[ 4 29 36  1 42]
[ 4 35 29 30  9]
[ 4 22 26 23 30]
[ 4 10 30 23  6]
[ 4 30 24 34 25]
[ 4  9  7  4 41]
[ 4 36 29 35 42]
[ 4 29 28  7  1]
[ 4 31 40 38  0]
[ 4 36  0 42 40]
[ 4 31 20 21 16]
[ 4 10 30  6  9]
[ 4 30 35 37 34]
[ 4 15 35 41 37]
[ 4 30  9 14 15]
[ 4 28 26 36 29]
[ 4  2  0 36 40]
[ 4 28 27 36 26]
[ 4 38 31  4  8]
[ 4 26 27 36 17]
[ 4 27  2 36 17]
[ 4 16 33 36 17]
[ 4 16 40 36 33]
[ 4 37 30 14 15]
[ 4 17 18 36 26]
[ 4 17 33 36 18]
[ 4  2 16 36 17]
[ 4  2 40 36 16]
[ 4 20 10 21

In [20]:
# Define the two functions here...
from bisect import bisect_right

import numpy as np


def check_points_bi(num_points, points):
    # Calculate point ID ranges for each object
    ranges = []
    curr_id = 0
    for num_point in num_points:
        ranges.append(curr_id + num_point)
        curr_id += num_point

    # Check which objects the points belong to
    point_objects = set()
    for point in points:
        # Binary search to find the object this point belongs to
        i = bisect_right(ranges, point)
        point_objects.add(i)

    # If more than one object is found, return False
    return len(point_objects) <= 1


def check_points_np(num_points, points):
    # Get a numpy array of the number of points for each object

    # Calculate the cumulative sum to get the ranges
    ranges = np.cumsum(num_points)

    # Check which objects the points belong to
    point_objects = np.searchsorted(ranges, points, side='right')

    # If more than one object is found, return False
    return len(set(point_objects)) <= 1


def check_points_np_unique(num_points, points):
    # Get a numpy array of the number of points for each object

    # Calculate the cumulative sum to get the ranges
    ranges = np.cumsum(num_points)

    # Check which objects the points belong to
    point_objects = np.searchsorted(ranges, points, side='right')

    # If more than one object is found, return False
    return len(np.unique(point_objects)) <= 1


# Generate some test data
num_objects = 1000000
num_points = np.random.randint(1, 10, size=num_objects)
points = np.random.randint(0, num_objects * 5, size=4)

In [24]:
%%timeit
check_points_bi(num_points, points)

91.7 ms ± 589 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit
check_points_np(num_points, points)

2.38 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%%timeit
check_points_np_unique(num_points, points)

2.47 ms ± 89.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
num_objects = 30
num_points = np.random.randint(1, 10, size=num_objects)
points = np.random.randint(0, num_objects * 5, size=4)

In [30]:
%%timeit
check_points_bi(num_points, points)

3.94 µs ± 40.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
%%timeit
check_points_np(num_points, points)

2.62 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [32]:
%%timeit
check_points_np_unique(num_points, points)

4.47 µs ± 23.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
